# DeepDream

Implemented By:
<table align="left">
  <td> <h5> <b> Roee Zamir </b> </h5> </td>
  <td>
    <a target="_blank" href="https://www.linkedin.com/in/roee-zamir-500121177/"><img src="https://cdn3.iconfinder.com/data/icons/free-social-icons/67/linkedin_circle_color-512.png" width="36"/></a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/roeez"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /></a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/roeez/DeepDream/blob/master/DeepDream.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>  
</table>



**What?**</br>
This notebook contains an unofficial minimal simplified PyTorch Implementation of [DeepDream](https://ai.googleblog.com/2015/07/deepdream-code-example-for-visualizing.html) (A. Mordvintsev 2015), a cool method to give your neural network LSD and create psychedelic images. 

**How?**</br>
The idea in DeepDream is very simple, we choose one or more layers of a trained network, and we modify the input image to "excite" these layers (increase their response), By doing so iteratively the network enhances patterns it sees in the image, resulting in a dream-like image.

**Why?**</br>
Networks just wanna have fun

![Example](https://github.com/roeez/DeepDream/raw/master/example/wis.gif)

---

####Imports, defintions and utility functions.####

In [0]:
import torch
import numpy
import math
import torchvision
import numpy as np
from torchvision import transforms as T
from PIL import Image
import IPython.display as display

class FeatureExtractor(torch.nn.Module):
    '''Get all the features extracted by the conv layers'''
    def __init__(self, model):
        super(FeatureExtractor, self).__init__()
        self.model = model
    
    def forward(self, x):
        feats = []
        for l in self.model.children():
            x = l(x)
            if isinstance(l, torch.nn.Conv2d):
                feats.append(x)
        return feats

class Clip(object):
    '''Clip values transformation'''
    def __init__(self, min, max):
        self.min = min
        self.max = max

    def __call__(self, x):
        return x.clamp(self.min, self.max)

# ImageNet normalization
mean = torch.tensor([0.485, 0.456, 0.406])
std = torch.tensor([0.229, 0.224, 0.225])

preprocess = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=mean, std=std),
])

postprocess = T.Compose([
    T.Normalize(mean=-mean/std, std=1/std),
    Clip(0,1),
    T.ToPILImage(),
])

def get_ranges(image):
    '''
    Returns the ranges [min_val, max_val] of the intensities of each channel
    of the input image (Used later for clipping of the modified image intensities)
    '''
    ranges = []
    for i in range(3):
        ranges.append([image[:,i,:].min().item(), image[:,i,:].max().item()])

    return ranges

def clip_intensities(image, ranges):
    for i in range(3):
        image[:,i,:,:] = image[:,i,:,:].clamp(*ranges[i])

    return image

def imshow(img):
  display.display(Image.fromarray(np.array(img)))

Now we will choose a trained classification model.</br>
`torchvision` offers a nice [list](https://pytorch.org/docs/stable/torchvision/models.html#classification) of implemented and trained models.<br>
Colab allows you to use GPU, use it! (Runtime->Change runtime type->GPU)

In [0]:
# Dismiss the fully connected layers
model = next(torchvision.models.vgg19(pretrained=True).children())
model = FeatureExtractor(model)
model.eval()

# In this method, we don't change the model's weights. 
for param in model.parameters():
    param.requires_grad = False

if torch.cuda.is_available():
    model.cuda()

Download and open an example image to work with

In [0]:
!wget https://github.com/roeez/DeepDream/raw/master/example/wis.jpg

IMG_PATH = '/content/wis.jpg'

input_batch = preprocess(Image.open(IMG_PATH))[None, ...]

if torch.cuda.is_available():
    input_batch = input_batch.cuda()


# The core code #
The original basic DeepDream algorithm takes as input a trained model, an input image, a chosen layer to extract features from, #iterations and learning rate. In this implementation i added suuport for a weighted multiple layers choice. Deeper layers respond to higher-level features as eyes or faces while earlier layers respond to low-level features as edges, shapes and textures.
In each iteration we calculate the strength of the response of the chosen layers to the input image ($\ell{_2}$ norm for e.g.) which will be increased using *gradient* ***ascent*** by calculating the gradient of the response with respect to the input image and adding it to the image.

In [0]:
def deepDream(model, image, layers, weights, iterations=10, lr=.01):
    ''' The core of DeepDream
    Params:
    model (nn.Module)
    image (Tensor)
    layers (list): list of chosen layers number
    weights (list): list of the weights of the layers
    iterations (int)
    lr (float): the learning rate
    '''

    ranges = get_ranges(image)

    # Gradients calculated with respect to the input image
    image = torch.autograd.Variable(image.clone(), requires_grad=True)

    # iteratively increase the response of the chosen layers 
    for i in range(iterations):
        # extract features from the selected layers
        feats = map(model(image).__getitem__, layers)

        loss = .0 # The strength of the layers' response to increase
        for w, f in zip(weights, feats):
            loss += w * f.norm() # You can try other response reductions
        loss.backward()

        # Normalize the gradients
        grad = image.grad.data - image.grad.data.mean()
        grad = grad / grad.std()

        # Gradient ascent - we want to increase the response
        image.data += lr * image.grad / image.grad.data.abs().mean()

        clip_intensities(image.data, ranges)

        image.grad.zero_()

    return image.data

In order to produce multi-scale fine patterns we will apply the algorithm on a pyramid - multiple scales of the input image (referred to as octaves by the author)

In [0]:
image = input_batch.clone()
original_size = torch.tensor(image.shape[-2:])
SCALE_FACTOR = 1.4
MIN_SCALE_POWER = -(np.floor(math.log(min(image.shape[-2:])/28, SCALE_FACTOR)).astype(np.int))
MAX_SCALE_POWER = 1

octaves = []

for i in range(MIN_SCALE_POWER, MAX_SCALE_POWER):
    octaves.append(torch.nn.functional.interpolate(image, size=(original_size*(SCALE_FACTOR**i)).int().tolist(), mode='bicubic'))

details = torch.zeros_like(octaves[0])
for octave_num, octave in enumerate(octaves):
    image = octave + torch.nn.functional.interpolate(details, size=octave.shape[-2:],  mode='bicubic')
    image = deepDream(model, image, [5, 7, 10], [.2, .2, .6], 20, .005)
    details = image - octave
    display.clear_output(wait=True)
    imshow(postprocess(image[0].cpu()))

image = torch.nn.functional.interpolate(image, size=original_size.int().tolist(), mode='bicubic')
display.clear_output(wait=True)
imshow(postprocess(image[0].cpu()))

# What's next? #
Feel free to try different images, models, layers, weights, #iterations, learning rate etc.
This implementation ommits some tricks that were used in the [original implementation](https://github.com/google/deepdream/blob/master/dream.ipynb) as random jittering to the input image and other tricks which i didn't find so useful. Another intresting suggestion is to use as objective the dot product the features of the image with features of other guide image to somewhat control the patterns.
# Takeaways#
*   Trained networks encapsulates patterns
*   Deeper layers reponse to high-level features
*   Earlier layers respose to low-level features

## Links ##


*   [Original blog post](https://ai.googleblog.com/2015/07/deepdream-code-example-for-visualizing.html)
*   [Original blog post #2](https:///ai.googleblog.com/2015/06/inceptionism-going-deeper-into-neural.html)
*   [Official Caffe implementation](https://github.com/google/deepdream)
*   [Official TensorFlow tutorial](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/generative/deepdream.ipynb)



